# Importing libraries

In [1]:
from pyzotero import zotero
import os
import tweepy as tw
import pandas as pd
import datetime
import json, sys
from datetime import date, timedelta  
import datetime
import plotly.express as px
import pycountry
import re
import pandas as pd
import requests
import time
from datetime import datetime, timedelta

In [2]:
"# All items in the Zotero Intelligence bibliography library".index("All")

2

In [3]:
library_id = '2514686'
library_type = 'group'
api_key = '' # api_key is only needed for private groups and libraries
zot = zotero.Zotero(library_id, library_type)


# All items in the Zotero Intelligence bibliography library

In [4]:
items = zot.everything(zot.top())

data3=[]
columns3=['Title','Publication type', 'Link to publication', 'Abstract', 'Zotero link', 'Date published', 'FirstName2', 'Publisher', 'Journal', 'Date added', 'Col key', 'DOI']

for item in items:
    # Extracting various types of creators
    authors = ""
    if 'creators' in item['data']:
        authors_info = item['data']['creators']
        author_names = []
        for author in authors_info:
            creator_type = author.get('creatorType')
            valid_types = ['author', 'contributor', 'editor', 'guest', 'podcaster', 'presenter', 'translator']
            
            if item['data']['itemType'] == 'bookSection' and creator_type == 'editor':
                continue  # Skip adding 'editor' for Book chapters
            
            if creator_type in valid_types:
                if 'firstName' in author and 'lastName' in author:
                    author_names.append(author['firstName'] + ' ' + author['lastName'])
                elif 'name' in author:
                    author_names.append(author['name'])
                
        authors = ', '.join(author_names)
        
    data3.append((
        item['data']['title'], 
        item['data']['itemType'], 
        item['data']['url'], 
        item['data']['abstractNote'], 
        item['links']['alternate']['href'],
        item['data'].get('date'),
        authors,  # Add the concatenated authors string
        item['data'].get('publisher'),
        item['data'].get('publicationTitle'),
        item['data']['dateAdded'],
        item['data']['collections'],
        item['data'].get('DOI')
        )) 
# pd.set_option('display.max_colwidth', None)

df = pd.DataFrame(data3, columns=columns3)

mapping_types = {
    'thesis': 'Thesis',
    'journalArticle': 'Journal article',
    'book': 'Book',
    'bookSection': 'Book chapter',
    'blogPost': 'Blog post',
    'videoRecording': 'Video',
    'podcast': 'Podcast',
    'magazineArticle': 'Magazine article',
    'webpage': 'Webpage',
    'newspaperArticle': 'Newspaper article',
    'report': 'Report',
    'forumPost': 'Forum post',
    'manuscript': 'Manuscript',
    'document': 'Document',
    'conferencePaper': 'Conference paper',
    'film': 'Film',
    'presentation': 'Presentation',
    'audioRecording':'Podcast',
    'preprint':'Preprint',
    'hearing':'Hearing'
}
df['Publication type'] = df['Publication type'].replace(mapping_types)

mapping_publisher = {
    'Taylor & Francis Group': 'Taylor and Francis',
    'Taylor and Francis': 'Taylor and Francis',
    'Taylor & Francis': 'Taylor and Francis',
    'Routledge': 'Routledge',
    'Routledge Handbooks Online': 'Routledge',
    'Praeger Security International': 'Praeger',
    'Praeger': 'Praeger'
}
df['Publisher'] = df['Publisher'].replace(mapping_publisher)

# df['Publisher'] = df['Publisher'].replace(['Taylor & Francis Group', 'Taylor and Francis', 'Taylor & Francis'], 'Taylor and Francis')
# df['Publisher'] = df['Publisher'].replace(['Routledge', 'Routledge Handbooks Online'], 'Routledge')
# df['Publisher'] = df['Publisher'].replace(['Praeger Security International', 'Praeger'], 'Praeger')

mapping_journal = {
    'International Journal of Intelligence and Counterintelligence': 'Intl Journal of Intelligence and Counterintelligence',
    'International Journal of Intelligence and CounterIntelligence': 'Intl Journal of Intelligence and Counterintelligence',
    'Intelligence and national security': 'Intelligence and National Security',
    'Intelligence and National Security': 'Intelligence and National Security',
    'Intelligence & National Security': 'Intelligence and National Security'
}

df['Journal'] = df['Journal'].replace(mapping_journal)

# df['Journal'] = df['Journal'].replace(['International Journal of Intelligence and Counterintelligence', 'International Journal of Intelligence and CounterIntelligence'], 'Intl Journal of Intelligence and Counterintelligence')
# df['Journal'] = df['Journal'].replace(['Intelligence and national security', 'Intelligence and National Security', 'Intelligence & National Security'], 'Intelligence and National Security')

In [5]:
# items = zot.everything(zot.top())

# data3=[]
# columns3=['Title','Publication type', 'Link to publication', 'Abstract', 'Zotero link', 'Date published', 'FirstName2', 'Publisher', 'Journal', 'Date added', 'Col key']

# for item in items:
#     # Extracting various types of creators
#     authors = ""
#     if 'creators' in item['data']:
#         authors_info = item['data']['creators']
#         author_names = []
#         for author in authors_info:
#             creator_type = author.get('creatorType')
#             valid_types = ['author', 'contributor', 'editor', 'guest', 'podcaster']
#             if creator_type in valid_types:  # Include all identified creator types
#                 if 'firstName' in author and 'lastName' in author:
#                     author_names.append(author['firstName'] + ' ' + author['lastName'])
#                 elif 'name' in author:
#                     author_names.append(author['name'])  # Handling cases where 'firstName' and 'lastName' are absent
                
#         authors = ', '.join(author_names)
        
#     data3.append((
#         item['data']['title'], 
#         item['data']['itemType'], 
#         item['data']['url'], 
#         item['data']['abstractNote'], 
#         item['links']['alternate']['href'],
#         item['data'].get('date'),
#         authors,  # Add the concatenated authors string
#         item['data'].get('publisher'),
#         item['data'].get('publicationTitle'),
#         item['data']['dateAdded'],
#         item['data']['collections']
#         )) 
# # pd.set_option('display.max_colwidth', None)

# df = pd.DataFrame(data3, columns=columns3)

# mapping_types = {
#     'thesis': 'Thesis',
#     'journalArticle': 'Journal article',
#     'book': 'Book',
#     'bookSection': 'Book chapter',
#     'blogPost': 'Blog post',
#     'videoRecording': 'Video',
#     'podcast': 'Podcast',
#     'magazineArticle': 'Magazine article',
#     'webpage': 'Webpage',
#     'newspaperArticle': 'Newspaper article',
#     'report': 'Report',
#     'forumPost': 'Forum post',
#     'manuscript': 'Manuscript',
#     'document': 'Document',
#     'conferencePaper': 'Conference paper',
#     'film': 'Film',
#     'presentation': 'Presentation',
#     'audioRecording':'Podcast'
# }
# df['Publication type'] = df['Publication type'].replace(mapping_types)

# mapping_publisher = {
#     'Taylor & Francis Group': 'Taylor and Francis',
#     'Taylor and Francis': 'Taylor and Francis',
#     'Taylor & Francis': 'Taylor and Francis',
#     'Routledge': 'Routledge',
#     'Routledge Handbooks Online': 'Routledge',
#     'Praeger Security International': 'Praeger',
#     'Praeger': 'Praeger'
# }
# df['Publisher'] = df['Publisher'].replace(mapping_publisher)

# # df['Publisher'] = df['Publisher'].replace(['Taylor & Francis Group', 'Taylor and Francis', 'Taylor & Francis'], 'Taylor and Francis')
# # df['Publisher'] = df['Publisher'].replace(['Routledge', 'Routledge Handbooks Online'], 'Routledge')
# # df['Publisher'] = df['Publisher'].replace(['Praeger Security International', 'Praeger'], 'Praeger')

# mapping_journal = {
#     'International Journal of Intelligence and Counterintelligence': 'Intl Journal of Intelligence and Counterintelligence',
#     'International Journal of Intelligence and CounterIntelligence': 'Intl Journal of Intelligence and Counterintelligence',
#     'Intelligence and national security': 'Intelligence and National Security',
#     'Intelligence and National Security': 'Intelligence and National Security',
#     'Intelligence & National Security': 'Intelligence and National Security'
# }

# df['Journal'] = df['Journal'].replace(mapping_journal)

# # df['Journal'] = df['Journal'].replace(['International Journal of Intelligence and Counterintelligence', 'International Journal of Intelligence and CounterIntelligence'], 'Intl Journal of Intelligence and Counterintelligence')
# # df['Journal'] = df['Journal'].replace(['Intelligence and national security', 'Intelligence and National Security', 'Intelligence & National Security'], 'Intelligence and National Security')

In [6]:
def zotero_collections2(library_id, library_type):
    collections = zot.collections()
    data = [(item['data']['key'], item['data']['name'], item['meta']['numItems'], item['links']['alternate']['href']) for item in collections]
    df_collections = pd.DataFrame(data, columns=['Key', 'Name', 'Number', 'Link'])
    return df_collections
df_collections_2 = zotero_collections2(library_id, library_type)

In [7]:
def duplicate_rows_by_col_key(df, df_collections):
    # Duplicate rows based on 'Col key'
    duplicated_rows = []
    for index, row in df.iterrows():
        if isinstance(row['Col key'], list):
            for key in row['Col key']:
                new_row = row.copy()
                collection_info = df_collections[df_collections['Key'] == key]
                if not collection_info.empty:
                    new_row['Collection_Key'] = key
                    new_row['Collection_Name'] = collection_info.iloc[0]['Name']
                    new_row['Collection_Link'] = collection_info.iloc[0]['Link']
                    duplicated_rows.append(new_row)
        else:
            key = row['Col key']
            new_row = row.copy()
            collection_info = df_collections[df_collections['Key'] == key]
            if not collection_info.empty:
                new_row['Collection_Key'] = key
                new_row['Collection_Name'] = collection_info.iloc[0]['Name']
                new_row['Collection_Link'] = collection_info.iloc[0]['Link']
                duplicated_rows.append(new_row)

    # Create a new DataFrame with duplicated rows
    duplicated_df = pd.DataFrame(duplicated_rows)

    return duplicated_df

# Duplicating rows based on 'Col key' and collections information
duplicated_df = duplicate_rows_by_col_key(df, df_collections_2)

In [8]:
# day_allitems = datetime.date.today().isoformat()

In [9]:
# df_fa = df['FirstName2']
# df_fa = pd.DataFrame(df_fa.tolist())
# df_fa = df_fa[0]
# df_fa = df_fa.apply(lambda x: {} if pd.isna(x) else x)
# df_new = pd.json_normalize(df_fa, errors='ignore') 
# df = pd.concat([df, df_new], axis=1)
# df['firstName'] = df['firstName'].fillna('null')
# df['lastName'] = df['lastName'].fillna('null')

In [10]:
df.to_csv('all_items.csv')
duplicated_df.to_csv('all_items_duplicated.csv')

In [11]:
import psycopg2
from sqlalchemy import create_engine
import psycopg2.extras as extras

# Countries 

In [12]:
# Read the csv file into a pandas dataframe
df = pd.read_csv('https://raw.githubusercontent.com/YusufAliOzkan/zotero-intelligence-bibliography/main/all_items.csv')

# Dictionary to map non-proper country names to their proper names
country_map = {
    'british': 'UK',
    'great britain': 'UK',
    'UK' : 'UK', 
    'america' : 'United States',
    'United States of America' : 'United States',
    'Soviet Union': 'Russia', 
    'american' : 'United States',
    'United States' : 'United States',
    'russian' : 'Russia'
    # Add more mappings as needed
}

# Find the country names in the "title" column of the dataframe
found_countries = {}
for i, row in df.iterrows():
    title = str(row['Title']).lower()
    for country in pycountry.countries:
        name = country.name.lower()
        if name in title or (name + 's') in title:  # Check for singular and plural forms of country names
            proper_name = country.name
            found_countries[proper_name] = found_countries.get(proper_name, 0) + 1
    for non_proper, proper in country_map.items():
        if non_proper in title:
            found_countries[proper] = found_countries.get(proper, 0) + title.count(non_proper)

# Create a new dataframe containing the found countries and their counts
df_countries = pd.DataFrame({'Country': list(found_countries.keys()), 'Count': list(found_countries.values())})

In [13]:
fig = px.choropleth(df_countries, locations='Country', locationmode='country names', color='Count', 
                    title='Country mentions in titles', color_continuous_scale='Viridis',
                    width=1100, height=700) # Adjust the size of the map here

# Display the map
fig.show()

In [14]:
df_countries=df_countries.sort_values(by='Count', ascending=False)
df_countries.reset_index(drop=True)

,Country,Count
0,United States,589
1,UK,435
2,Israel,137
3,Russia,124
4,Ukraine,101
...,...,...
103,Tajikistan,1
104,Costa Rica,1
105,Estonia,1
106,El Salvador,1


In [15]:
fig = px.choropleth(df_countries, locations='Country', locationmode='country names', color='Count', 
                    title='Country mentions in titles', color_continuous_scale='Viridis',
                    width=1100, height=700) # Adjust the size of the map here

# Display the map
fig.show()

In [16]:
df_countries.to_csv('countries.csv',index=False)

# NER analysis

In [17]:
import spacy
import nltk
import ast
from spacy.pipeline import EntityRecognizer

In [18]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
df = pd.read_csv('https://raw.githubusercontent.com/YusufAliOzkan/zotero-intelligence-bibliography/main/all_items.csv')
nlp = spacy.load("en_core_web_sm")
ruler = nlp.add_pipe("entity_ruler")
patterns = [{"label": "ORG", "pattern": "MI6"}]
ruler.add_patterns(patterns)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yaozk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\yaozk\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\yaozk\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\yaozk\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
c:\Users\yaozk\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\util.py:865: UserWarning:

[W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible

In [19]:
def extract_entities(text):
    doc = nlp(text)
    orgs = []
    gpes = []
    people = []
    for entity in doc.ents:
        if entity.label_ == 'ORG':
            orgs.append(entity.text)
        elif entity.label_ == 'GPE':
            gpes.append(entity.text)
        elif entity.label_ == 'PERSON':
            people.append(entity.text)
    return pd.Series({'ORG': orgs, 'GPE': gpes, 'PERSON': people})

In [20]:
df_title = df[['Title']].copy()
df_title = df_title.rename(columns={'Title':'Text'})
df_abstract = df[['Abstract']].copy()
df_abstract = df_abstract.rename(columns={'Abstract':'Text'})
df_one = pd.concat([df_title, df_abstract], ignore_index=True)
df_one['Text'] = df_one['Text'].fillna('')

In [21]:
df_one = pd.concat([df_one[['Text']], df_one['Text'].apply(extract_entities)], axis=1)
df_one = df_one.explode('GPE').reset_index(drop=True)
df_one = df_one.explode('ORG').reset_index(drop=True)
df_one = df_one.explode('PERSON').reset_index(drop=True)

In [22]:
df_one_g = df_one.copy()
df_one_g = df_one[['Text', 'GPE']]
# df_one_g = df_one_g.fillna('')
df_one_g = df_one_g.drop_duplicates(subset=['Text', 'GPE'])

gpe_counts = df_one_g['GPE'].value_counts().reset_index()
gpe_counts.columns = ['GPE', 'count']

# pd.options.display.max_rows = None


mapping_locations = {
    'the United States': 'USA',
    'The United States': 'USA',
    'US': 'USA',
    'U.S.': 'USA',
    'United States' : 'USA',
    'America' : 'USA',
    'the United States of America' : 'USA',
    'Britain' : 'UK',
    'the United Kingdom': 'UK',
    'U.K.' : 'UK',
    'Global Britain' : 'UK',
    'United Kingdom' : 'UK', 
    'the Soviet Union' : 'Russia',
    'The Soviet Union' : 'Russia',
    'USSR' : 'Russia',
    'Ukraine - Perspective' : 'Ukraine',
    'Ukrainian' : 'Ukraine',
    'Great Britain' : 'UK',
    'Ottoman Empire' : 'Turkey'
}
gpe_counts['GPE'] =gpe_counts['GPE'].replace(mapping_locations)
gpe_counts = gpe_counts.groupby('GPE').sum().reset_index()
gpe_counts.sort_values('count', ascending=False, inplace=True)
gpe_counts = gpe_counts.reset_index(drop=True)
gpe_counts.head(15)

,GPE,count
0,USA,1211
1,UK,647
2,Russia,442
3,Germany,184
4,Ukraine,180
5,Israel,174
6,France,134
7,China,132
8,Canada,128
9,Iraq,124


In [23]:
df_one_p = df_one.copy()
df_one_p = df_one[['Text', 'PERSON']]
# df_one_p = df_one_g.fillna('')
df_one_p = df_one_p.drop_duplicates(subset=['Text', 'PERSON'])

person_counts = df_one_p['PERSON'].value_counts().reset_index()
person_counts.columns = ['PERSON', 'count']

mapping_person = {
    'Putin' : 'Vladimir Putin',
    'Vladimir Putin' : 'Vladimir Putin',
    'Churchill' : 'Winston Churchill',
    'Hitler' : 'Adolf Hitler',
    'Biden' : 'Joe Biden',
    "John le Carré’s" : "John le Carré"
}

person_counts['PERSON'] =person_counts['PERSON'].replace(mapping_person)
person_counts = person_counts.groupby('PERSON').sum().reset_index()
person_counts.sort_values('count', ascending=False, inplace=True)

remove_person = ['MI6', 'Twitter', 'GRU'
          ]
person_counts = person_counts[~person_counts['PERSON'].isin(remove_person)]
person_counts = person_counts.reset_index(drop=True)

person_counts.head(15)

,PERSON,count
0,Adolf Hitler,74
1,Vladimir Putin,69
2,Winston Churchill,45
3,Stalin,37
4,Mujibur Rahman,28
5,Bush,26
6,James Bond,23
7,John le Carré,23
8,Eisenhower,22
9,Brexit,18


In [24]:
df_one_o = df_one.copy()
df_one_o = df_one[['Text', 'ORG']]
# df_one_p = df_one_g.fillna('')
df_one_o = df_one_o.drop_duplicates(subset=['Text', 'ORG'])

org_counts = df_one_o['ORG'].value_counts().reset_index()
org_counts.columns = ['ORG', 'count']

mapping_organisations = {
    'The British Secret Intelligence Service' : 'SIS',
    'the British Secret Intelligence Service' : 'SIS',
    'The Joint Intelligence Committee' : 'Joint Intelligence Committee',
    'the Joint Intelligence Committee' : 'Joint Intelligence Committee',
    'Joint Intelligence Committee' : 'Joint Intelligence Committee',
    'the Joint Intelligence Committee - History' : 'Joint Intelligence Committee',
    'Central Intelligence Agency' : 'CIA',
    'the Central Intelligence Agency' : 'CIA',
    'the Foreign Office' : 'Foreign Office',
    'Schar School' : 'Schar School of Policy and Government',
    'the Secret Intelligence Service' : 'SIS',
    "George Mason University's" : "George Mason University",
    'JIC' : 'Joint Intelligence Committee'
}
org_counts['ORG'] =org_counts['ORG'].replace(mapping_organisations)
org_counts = org_counts.groupby('ORG').sum().reset_index()
org_counts.sort_values('count', ascending=False, inplace=True)

remove_orgs = ['Intelligence', 'Kremlin', 'Ultra', 'International Security', 'Intelligence Studies', 'Intelligence Analysis', 'OSINT']
org_counts = org_counts[~org_counts['ORG'].isin(remove_orgs)]
org_counts = org_counts.reset_index(drop=True)

org_counts.head(15)

,ORG,count
0,CIA,588
1,KGB,126
2,FBI,83
3,Allied,81
4,SIS,62
5,Congress,59
6,SOE,58
7,the Intelligence Community,53
8,NATO,52
9,EU,49


In [25]:
gpe_counts.head(15).to_csv('gpe.csv')
person_counts.head(15).to_csv('person.csv')
org_counts.head(15).to_csv('org.csv')

# Retrieving citation count and open access status

In [26]:
## Retrieving citation and open access status
df_doi = pd.read_csv('all_items.csv')
# df_doi = df_doi.head(10)
if datetime.today().weekday() == 3:

    df_doi = df_doi[['Zotero link', 'DOI']].dropna()
    df_doi = df_doi.drop(df_doi[df_doi['DOI']==''].index)
    df_doi = df_doi.reset_index(drop=True)
    df_doi['DOI'] = df_doi['DOI'].str.replace('https://doi.org/', '')
    df_doi

    def fetch_article_metadata(doi):
        base_url = 'https://api.openalex.org/works/https://doi.org/'
        response = requests.get(base_url + doi)
        if response.status_code == 200:
            data = response.json()
            article_metadata = {
                'ID': data.get('id'),
                'Citation': data.get('cited_by_count'),
                'OA status': data.get('open_access', {}).get('is_oa'),
                'Citation_list':data.get('cited_by_api_url')
            }
            return article_metadata
        else:
            return {'ID': None, 'Citation': None, 'OA status': None}

    df_doi['ID'] = None
    df_doi['Citation'] = None
    df_doi['OA status'] = None
    df_doi['Citation_list'] = None

    # Iterate over each row in the DataFrame
    for index, row in df_doi.iterrows():
        doi = row['DOI']
        article_metadata = fetch_article_metadata(doi)
        if article_metadata:
            # Update DataFrame with fetched information
            df_doi.at[index, 'ID'] = article_metadata['ID']
            df_doi.at[index, 'Citation'] = article_metadata['Citation']
            df_doi.at[index, 'OA status'] = article_metadata['OA status']
            if 'Citation_list' in article_metadata:
                df_doi.at[index, 'Citation_list'] = article_metadata['Citation_list']
            else:
                # Set to None if key is not found
                df_doi.at[index, 'Citation_list'] = None
    df_doi
    df_doi.to_csv('citations.csv')
else:
    print('This part will be refreshed every Wednesday!')

C:\Users\yaozk\AppData\Local\Temp\ipykernel_20400\1323847147.py:9: FutureWarning:

The default value of regex will change from True to False in a future version.



In [27]:
df_doi

,Zotero link,DOI,ID,Citation,OA status,Citation_list
0,https://www.zotero.org/groups/intelligence_bib...,10.1177/1940161214566693,https://openalex.org/W2329812692,16,True,https://api.openalex.org/works?filter=cites:W2...
1,https://www.zotero.org/groups/intelligence_bib...,10.1177/0896920518780987,https://openalex.org/W2884816909,2,True,https://api.openalex.org/works?filter=cites:W2...
2,https://www.zotero.org/groups/intelligence_bib...,10.1111/1467-9256.00138,https://openalex.org/W1993050611,151,False,https://api.openalex.org/works?filter=cites:W1...
3,https://www.zotero.org/groups/intelligence_bib...,10.1007/s12117-000-1002-2,https://openalex.org/W2032501189,3,False,https://api.openalex.org/works?filter=cites:W2...
4,https://www.zotero.org/groups/intelligence_bib...,10.1080/16161262.2016.1145854,https://openalex.org/W2308982232,4,False,https://api.openalex.org/works?filter=cites:W2...
...,...,...,...,...,...,...
3276,https://www.zotero.org/groups/intelligence_bib...,10.1177/1755088217712069,https://openalex.org/W2612406272,4,True,https://api.openalex.org/works?filter=cites:W2...
3277,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2019.1660951,https://openalex.org/W2996219269,5,False,https://api.openalex.org/works?filter=cites:W2...
3278,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2012.705186,https://openalex.org/W2033999438,26,False,https://api.openalex.org/works?filter=cites:W2...
3279,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2020.1748993,https://openalex.org/W3022421216,0,False,https://api.openalex.org/works?filter=cites:W3...


In [28]:
df_doi_2 = pd.read_csv('citations.csv', usecols=lambda column: column != 'Unnamed: 0')
df_doi_2

,Zotero link,DOI,ID,Citation,OA status,Citation_list
0,https://www.zotero.org/groups/intelligence_bib...,10.1177/1940161214566693,https://openalex.org/W2329812692,16.0,True,https://api.openalex.org/works?filter=cites:W2...
1,https://www.zotero.org/groups/intelligence_bib...,10.1177/0896920518780987,https://openalex.org/W2884816909,2.0,True,https://api.openalex.org/works?filter=cites:W2...
2,https://www.zotero.org/groups/intelligence_bib...,10.1111/1467-9256.00138,https://openalex.org/W1993050611,151.0,False,https://api.openalex.org/works?filter=cites:W1...
3,https://www.zotero.org/groups/intelligence_bib...,10.1007/s12117-000-1002-2,https://openalex.org/W2032501189,3.0,False,https://api.openalex.org/works?filter=cites:W2...
4,https://www.zotero.org/groups/intelligence_bib...,10.1080/16161262.2016.1145854,https://openalex.org/W2308982232,4.0,False,https://api.openalex.org/works?filter=cites:W2...
...,...,...,...,...,...,...
3276,https://www.zotero.org/groups/intelligence_bib...,10.1177/1755088217712069,https://openalex.org/W2612406272,4.0,True,https://api.openalex.org/works?filter=cites:W2...
3277,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2019.1660951,https://openalex.org/W2996219269,5.0,False,https://api.openalex.org/works?filter=cites:W2...
3278,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2012.705186,https://openalex.org/W2033999438,26.0,False,https://api.openalex.org/works?filter=cites:W2...
3279,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2020.1748993,https://openalex.org/W3022421216,0.0,False,https://api.openalex.org/works?filter=cites:W3...


In [29]:
df_1 = pd.read_csv('all_items.csv', usecols=lambda column: column != 'Unnamed: 0')
df_doi_1 = pd.read_csv('citations.csv', usecols=lambda column: column != 'Unnamed: 0')
df_doi_1 = df_doi_1.drop(columns=['DOI'])
df_1 = pd.merge(df_1, df_doi_1, on='Zotero link', how='left')
df_1.to_csv('all_items.csv')

In [30]:
df_2 = pd.read_csv('all_items_duplicated.csv', usecols=lambda column: column != 'Unnamed: 0')
df_doi_2 = pd.read_csv('citations.csv', usecols=lambda column: column != 'Unnamed: 0')
df_doi_2 = df_doi_2.drop(columns=['DOI'])
df_2 = pd.merge(df_2, df_doi_2, on='Zotero link', how='left')
df_2.to_csv('all_items_duplicated.csv')

In [31]:
df_2 = pd.read_csv('all_items_duplicated.csv')
df_2

,Unnamed: 0,Title,Publication type,Link to publication,Abstract,Zotero link,Date published,FirstName2,Publisher,Journal,Date added,Col key,DOI,Collection_Key,Collection_Name,Collection_Link,ID,Citation,OA status,Citation_list
0,0,United States,Book chapter,https://doi.org/10.1007/978-3-031-47581-8_8,This chapter explores a brief history of the U...,https://www.zotero.org/groups/intelligence_bib...,2024-02-12,"John Michael Weaver, Tom Røseth",Springer International Publishing,NaN,2024-02-15T09:49:34Z,['T92JK7A5'],NaN,T92JK7A5,07.2 SIGINT,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN
1,1,United Kingdom (UK),Book chapter,https://doi.org/10.1007/978-3-031-47581-8_7,This chapter explores a brief history of the U...,https://www.zotero.org/groups/intelligence_bib...,2024-02-12,"John Michael Weaver, Tom Røseth",Springer International Publishing,NaN,2024-02-15T09:49:21Z,['T92JK7A5'],NaN,T92JK7A5,07.2 SIGINT,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN
2,2,New Zealand,Book chapter,https://doi.org/10.1007/978-3-031-47581-8_6,This chapter explores a brief history of New Z...,https://www.zotero.org/groups/intelligence_bib...,2024-02-12,"John Michael Weaver, Tom Røseth",Springer International Publishing,NaN,2024-02-15T09:49:05Z,"['T92JK7A5', 'AZ3BZ9BR']",NaN,T92JK7A5,07.2 SIGINT,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN
3,3,New Zealand,Book chapter,https://doi.org/10.1007/978-3-031-47581-8_6,This chapter explores a brief history of New Z...,https://www.zotero.org/groups/intelligence_bib...,2024-02-12,"John Michael Weaver, Tom Røseth",Springer International Publishing,NaN,2024-02-15T09:49:05Z,"['T92JK7A5', 'AZ3BZ9BR']",NaN,AZ3BZ9BR,14 Global intelligence,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN
4,4,Canada,Book chapter,https://doi.org/10.1007/978-3-031-47581-8_5,This chapter explores a brief history of Canad...,https://www.zotero.org/groups/intelligence_bib...,2024-02-12,"John Michael Weaver, Tom Røseth",Springer International Publishing,NaN,2024-02-15T09:48:48Z,"['T92JK7A5', 'AZ3BZ9BR']",NaN,T92JK7A5,07.2 SIGINT,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8679,8679,Intelligence and Government in Britain and the...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2012,Philip H.J. Davies,Praeger,NaN,2020-06-05T10:17:21Z,"['HCN8YFI8', 'DVEM4H4W']",NaN,DVEM4H4W,"06 Accountability, oversight, and ethics",https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN
8680,8680,Silent Warfare: Understanding the World of Int...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2002,"Abram N. Shulsky, Gary Schmitt",Potomac Books,NaN,2020-06-05T10:17:21Z,"['HCN8YFI8', 'DVEM4H4W']",NaN,HCN8YFI8,02 Intelligence studies,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN
8681,8681,Silent Warfare: Understanding the World of Int...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2002,"Abram N. Shulsky, Gary Schmitt",Potomac Books,NaN,2020-06-05T10:17:21Z,"['HCN8YFI8', 'DVEM4H4W']",NaN,DVEM4H4W,"06 Accountability, oversight, and ethics",https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN
8682,8682,Intelligence and Government in Britain and the...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2012,Philip H.J. Davies,Praeger,NaN,2020-06-05T10:17:21Z,"['HCN8YFI8', 'DVEM4H4W']",NaN,HCN8YFI8,02 Intelligence studies,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN
